## **This python notebook is used to prep data identify a loan defaulter/charge off vs fully paid loan status for the lending club loan dataset. **

---



In [0]:
# Import basic data science packages
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import time
%matplotlib inline

**Importing some standard python packages above and looking at the shape and data types of the dataset. Please upload the file into the file section before running then next code block.**

In [1095]:
df = pd.read_csv("loan.csv")
df.shape
#df.head(3)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(39717, 111)

In [1096]:
df.dtypes

id                                  int64
member_id                           int64
loan_amnt                           int64
funded_amnt                         int64
funded_amnt_inv                   float64
term                               object
int_rate                           object
installment                       float64
grade                              object
sub_grade                          object
emp_title                          object
emp_length                         object
home_ownership                     object
annual_inc                        float64
verification_status                object
issue_d                            object
loan_status                        object
pymnt_plan                         object
url                                object
desc                               object
purpose                            object
title                              object
zip_code                           object
addr_state                        

In [1097]:
df.describe()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,...,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,3.971700e+04,3.971700e+04,39717.000000,39717.000000,39717.000000,39717.000000,3.971700e+04,39717.000000,39717.000000,39717.000000,14035.000000,2786.000000,39717.000000,39717.000000,39717.000000,39717.000000,39717.000000,39717.000000,39717.000000,39717.000000,39717.000000,39717.000000,39717.000000,39717.000000,39717.000000,39717.000000,39661.0,0.0,39717.0,0.0,0.0,0.0,39717.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39661.0,39717.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39020.000000,39678.0,0.0,0.0,0.0,0.0
mean,6.831319e+05,8.504636e+05,11219.443815,10947.713196,10397.448868,324.561922,6.896893e+04,13.315130,0.146512,0.869200,35.900962,69.698134,9.294408,0.055065,13382.528086,22.088828,51.227887,50.989768,12153.596544,11567.149118,9793.348813,2263.663172,1.363015,95.221624,12.406112,2678.826162,0.0,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.043260,0.0,NaN,NaN,NaN,NaN
std,2.106941e+05,2.656783e+05,7456.670694,7187.238670,7128.450439,208.874874,6.379377e+04,6.678594,0.491812,1.070219,22.020060,43.822529,4.400282,0.237200,15885.016641,11.401709,375.172839,373.824457,9042.040766,8942.672613,7065.522127,2608.111964,7.289979,688.744771,148.671593,4447.136012,0.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.204324,0.0,NaN,NaN,NaN,NaN
min,5.473400e+04,7.069900e+04,500.000000,500.000000,0.000000,15.690000,4.000000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.0,NaN,NaN,NaN,NaN
25%,5.162210e+05,6.667800e+05,5500.000000,5400.000000,5000.000000,167.020000,4.040400e+04,8.170000,0.000000,0.000000,18.000000,22.000000,6.000000,0.000000,3703.000000,13.000000,0.000000,0.000000,5576.930000,5112.310000,4600.000000,662.180000,0.000000,0.000000,0.000000,218.680000,0.0,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.0,NaN,NaN,NaN,NaN
50%,6.656650e+05,8.508120e+05,10000.000000,9600.000000,8975.000000,280.220000,5.900000e+04,13.400000,0.000000,1.000000,34.000000,90.000000,9.000000,0.0000

**Column Clean up : Removing columns that  have more than 80% NA.**

In [1098]:
df.columns[df.isnull().mean() < 0.8]

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose',
       'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'policy_code', 'application_type',
       'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt',
       'pub_rec_bankruptcies', 'tax_liens'],
      dtype='object')

In [1099]:
df2 = df[df.columns[df.isnull().mean() < 0.8]]
df2.shape

(39717, 55)

In [0]:
length1 = len(df2)

**Checking for duplicate rows in the data. Printing the number of rows before and after removing duplicates. Looks like there are no duplicates.**

In [1101]:
# length after removing duplicates 
length2=len(df2) 
df2.drop_duplicates(keep=False,inplace=True) 
   
# printing all data frame lengths  
print(length1, length2) 

39717 39717


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


**Removing columns that have unrelevant data like url, application type etc by dropping them.**

In [0]:
df2 = df2.drop(['url','desc','policy_code','last_pymnt_d','earliest_cr_line','emp_title','id', 'title', 'total_rec_int','application_type','initial_list_status','pymnt_plan', 'total_rec_late_fee', 'total_rec_prncp', 'zip_code'], axis=1)

In [1103]:
df2.shape

(39717, 40)

**Converting the columns to the appropriate data type here by using the astype command**

In [0]:
for col in ['member_id', 'grade', 'sub_grade', 'home_ownership','emp_length','term','verification_status','loan_status','purpose','addr_state']:
    df2[col] = df2[col].astype('category')

In [0]:
for col in ['member_id', 'grade', 'sub_grade', 'home_ownership','emp_length','term','verification_status','loan_status','purpose','addr_state']:
    df2[col] = df2[col].astype('category')

In [1106]:
df2.dtypes

member_id                     category
loan_amnt                        int64
funded_amnt                      int64
funded_amnt_inv                float64
term                          category
int_rate                        object
installment                    float64
grade                         category
sub_grade                     category
emp_length                    category
home_ownership                category
annual_inc                     float64
verification_status           category
issue_d                         object
loan_status                   category
purpose                       category
addr_state                    category
dti                            float64
delinq_2yrs                      int64
inq_last_6mths                   int64
mths_since_last_delinq         float64
open_acc                         int64
pub_rec                          int64
revol_bal                        int64
revol_util                      object
total_acc                

**As Loan_status is the target variable, checking the distinct count of the values in the dataset. Looks like an imbalanced dataset.**

In [1107]:
df2.loan_status.value_counts()


Fully Paid     32950
Charged Off     5627
Current         1140
Name: loan_status, dtype: int64

**As there are two % fields with "%" in the data, the following code block removes the "%" from the cell and converts the columns to float datatype.**

In [0]:
for col in ['int_rate','revol_util']:
  df2[col].head()
  df2[col] = df2[col].str.rstrip("%\n ")
  df2[col].head()
  df2[col]=df2[col].astype(float)


In [1109]:
df2[['int_rate','revol_util']].describe()

,int_rate,revol_util
count,39717.000000,39667.000000
mean,12.021177,48.832152
std,3.724825,28.332634
min,5.420000,0.000000
25%,9.250000,25.400000
50%,11.860000,49.300000
75%,14.590000,72.400000
max,24.590000,99.900000


**The following code blocks look at the number of missing values, histogram of the values in the columns. It also checks for the correlation plot among the integer/float columns.**

**The following code block imputes the missing values in the selected data columns with the mean value. The numbeer of rows with NA's before and after imputation are printed out to verify.**

**Feature Engineering : Creating two columns days since credit pull and days since loan issue from the two date columns after doing some data type changes.**

In [0]:
import datetime

df2['issue_d']=df2['issue_d']+"-2018"
df2['last_credit_pull_d']=df2['last_credit_pull_d']+"-2018"

In [0]:
from datetime import datetime
from dateutil.parser import parse
#y=datetime.strptime((df2.last_credit_pull_d).astype(str), '%m-%d-%Y')
df2['issue_d']=pd.to_datetime(df2['issue_d'])
df2['last_credit_pull_d']=pd.to_datetime(df2['last_credit_pull_d'])

In [0]:
from datetime import date
from datetime import datetime

today = str(date.today())
today=pd.to_datetime(today)
b=(today-df2['issue_d']).astype(str)
b=(b.str.rstrip("days +00:00:00.000000000 "))
df2['days_since_issue']=(b.str.rstrip("NaT"))
df2['days_since_issue'].value_counts()
df2['days_since_issue']=pd.to_numeric(df2['days_since_issue'])
#df2['days_since_issue'].value_counts()

In [0]:
from datetime import date
from datetime import datetime

today = str(date.today())
today=pd.to_datetime(today)
b=(today-df2['last_credit_pull_d']).astype(str)
b=(b.str.rstrip("days +00:00:00.000000000 "))
df2['days_since_credit_pull']=(b.str.rstrip("NaT"))
df2['days_since_credit_pull'].value_counts()
df2['days_since_credit_pull']=pd.to_numeric(df2['days_since_credit_pull'])
#df2['days_since_credit_pull'].value_counts()

In [0]:
df2 = df2.drop(['issue_d','last_credit_pull_d'],axis=1)

In [1115]:
for col in ['days_since_credit_pull','days_since_issue','revol_util','int_rate','pub_rec_bankruptcies','collections_12_mths_ex_med','chargeoff_within_12_mths','tax_liens','mths_since_last_delinq']:
  print(col," number of na's before imputation",":",df2[col].isna().sum())
  mean_value=df2[col].mean()
  df2[col]=df2[col].fillna(mean_value)
  print(col," number of na's after imputation",":",df2[col].isna().sum())

days_since_credit_pull  number of na's before imputation : 2
days_since_credit_pull  number of na's after imputation : 0
days_since_issue  number of na's before imputation : 0
days_since_issue  number of na's after imputation : 0
revol_util  number of na's before imputation : 50
revol_util  number of na's after imputation : 0
int_rate  number of na's before imputation : 0
int_rate  number of na's after imputation : 0
pub_rec_bankruptcies  number of na's before imputation : 697
pub_rec_bankruptcies  number of na's after imputation : 0
collections_12_mths_ex_med  number of na's before imputation : 56
collections_12_mths_ex_med  number of na's after imputation : 0
chargeoff_within_12_mths  number of na's before imputation : 56
chargeoff_within_12_mths  number of na's after imputation : 0
tax_liens  number of na's before imputation : 39
tax_liens  number of na's after imputation : 0
mths_since_last_delinq  number of na's before imputation : 25682
mths_since_last_delinq  number of na's afte

In [1116]:
#mode_value=df2['emp_length'].mode()
#df2['emp_length']=df2['emp_length'].fillna(mode_value)
print("Emp Length number of na's before imputation",":",df2['emp_length'].isna().sum())
df2['emp_length'] = df2['emp_length'].fillna(df2['emp_length'].mode().iloc[0])
print("Emp Length number of na's after imputation",":",df2['emp_length'].isna().sum())

Emp Length number of na's before imputation : 1075
Emp Length number of na's after imputation : 0


In [1117]:
df2['emp_length'].value_counts()

10+ years    9954
< 1 year     4583
2 years      4388
3 years      4095
4 years      3436
5 years      3282
1 year       3240
6 years      2229
7 years      1773
8 years      1479
9 years      1258
Name: emp_length, dtype: int64

In [1118]:
print(df2.describe().transpose())
print(df2.dtypes)
print("Number of rows and columns in the data:", df2.shape)

                              count          mean  ...           75%           max
loan_amnt                   39717.0  11219.443815  ...  15000.000000  3.500000e+04
funded_amnt                 39717.0  10947.713196  ...  15000.000000  3.500000e+04
funded_amnt_inv             39717.0  10397.448868  ...  14400.000000  3.500000e+04
int_rate                    39717.0     12.021177  ...     14.590000  2.459000e+01
installment                 39717.0    324.561922  ...    430.780000  1.305190e+03
annual_inc                  39717.0  68968.926377  ...  82300.000000  6.000000e+06
dti                         39717.0     13.315130  ...     18.600000  2.999000e+01
delinq_2yrs                 39717.0      0.146512  ...      0.000000  1.100000e+01
inq_last_6mths              39717.0      0.869200  ...      1.000000  8.000000e+00
mths_since_last_delinq      39717.0     35.900962  ...     35.900962  1.200000e+02
open_acc                    39717.0      9.294408  ...     12.000000  4.400000e+01
pub_

In [1126]:
import pandas_profiling
report=pandas_profiling.ProfileReport(df2)
report

Number of variables,40
Number of observations,39717
Total Missing (%),0.0%
Total size in memory,11.4 MiB
Average record size in memory,302.2 B
Numeric,19
Categorical,9
Boolean,0
Date,0
Text (Unique),1
Rejected,11


In [1134]:
df3=df2.drop(['acc_now_delinq','funded_amnt','chargeoff_within_12_mths','collections_12_mths_ex_med','delinq_amnt','funded_amnt_inv','out_prncp','tax_liens','total_pymnt'],axis=1)
print(df3.shape)
df3.head()

(39717, 31)


,member_id,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp_inv,total_pymnt_inv,recoveries,collection_recovery_fee,last_pymnt_amnt,pub_rec_bankruptcies,days_since_issue,days_since_credit_pull
0,1296599,5000,36 months,10.65,162.87,B,B2,10+ years,RENT,24000.0,Verified,Fully Paid,credit_card,AZ,27.65,0,1,35.900962,3,0,13648,83.7,9,0.00,5833.84,0.00,0.00,171.62,0.0,283,492.0
1,1314167,2500,60 months,15.27,59.83,C,C4,< 1 year,RENT,30000.0,Source Verified,Charged Off,car,GA,1.00,0,5,35.900962,3,0,1687,9.4,4,0.00,1008.71,117.08,1.11,119.66,0.0,283,372.0
2,1313524,2400,36 months,15.96,84.33,C,C5,10+ years,RENT,12252.0,Not Verified,Fully Paid,small_business,IL,8.72,0,2,35.900962,2,0,2956,98.5,10,0.00,3005.67,0.00,0.00,649.91,0.0,283,492.0
3,1277178,10000,36 months,13.49,339.31,C,C1,10+ years,RENT,49200.0,Source Verified,Fully Paid,other,CA,20.00,0,1,35.000000,10,0,5598,21.0,37,0.00,12231.89,0.00,0.00,357.48,0.0,283,522.0
4,1311748,3000,60 months,12.69,67.79,B,B5,1 year,RENT,80000.0,Source Verified,Current,other,OR,17.94,0,0,38.000000,15,0,27783,53.9,38,524.06,3513.33,0.00,0.00,67.79,0.0,283,492.0


**Convert Categorical Variables to dummy variables by one-hot encoding and write out the prepared file to a csv.**

In [0]:
cat_columns = ["grade", "sub_grade","emp_length","home_ownership","verification_status","purpose","addr_state","term"]
df2_processed = pd.get_dummies(df3, prefix_sep="__",
                              columns=cat_columns)

In [1135]:
df2_processed.to_csv("ML.csv")
print(df2_processed.shape)
df2_processed.head()

(39717, 150)


,member_id,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp_inv,total_pymnt_inv,recoveries,collection_recovery_fee,last_pymnt_amnt,pub_rec_bankruptcies,days_since_issue,days_since_credit_pull,grade__A,grade__B,grade__C,grade__D,grade__E,grade__F,grade__G,sub_grade__A1,sub_grade__A2,sub_grade__A3,sub_grade__A4,sub_grade__A5,sub_grade__B1,sub_grade__B2,sub_grade__B3,sub_grade__B4,sub_grade__B5,...,addr_state__IA,addr_state__ID,addr_state__IL,addr_state__IN,addr_state__KS,addr_state__KY,addr_state__LA,addr_state__MA,addr_state__MD,addr_state__ME,addr_state__MI,addr_state__MN,addr_state__MO,addr_state__MS,addr_state__MT,addr_state__NC,addr_state__NE,addr_state__NH,addr_state__NJ,addr_state__NM,addr_state__NV,addr_state__NY,addr_state__OH,addr_state__OK,addr_state__OR,addr_state__PA,addr_state__RI,addr_state__SC,addr_state__SD,addr_state__TN,addr_state__TX,addr_state__UT,addr_state__VA,addr_state__VT,addr_state__WA,addr_state__WI,addr_state__WV,addr_state__WY,term__ 36 months,term__ 60 months
0,1296599,5000,10.65,162.87,24000.0,Fully Paid,27.65,0,1,35.900962,3,0,13648,83.7,9,0.00,5833.84,0.00,0.00,171.62,0.0,283,492.0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,1314167,2500,15.27,59.83,30000.0,Charged Off,1.00,0,5,35.900962,3,0,1687,9.4,4,0.00,1008.71,117.08,1.11,119.66,0.0,283,372.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,1313524,2400,15.96,84.33,12252.0,Fully Paid,8.72,0,2,35.900962,2,0,2956,98.5,10,0.00,3005.67,0.00,0.00,649.91,0.0,283,492.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,1277178,10000,13.49,339.31,49200.0,Fully Paid,20.00,0,1,35.000000,10,0,5598,21.0,37,0.00,12231.89,0.00,0.00,357.48,0.0,283,522.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,1311748,3000,12.69,67.79,80000.0,Current,17.94,0,0,38.000000,15,0,27783,53.9,38,524.06,3513.33,0.00,0.00,67.79,0.0,283,492.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
